In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from utils_subdivision.gen_distribution_single_plots import analyze_phases
from utils_subdivision.gen_distribution_subplot import analyze_single_type    # plot_combined_results
from utils_subdivision.gen_distribution_merged_plot import plot_merged

In [20]:
mode = ["group", "individual", "audience"]
m_idx = 1

with open('data/selected_piece_list.pkl', 'rb') as f:
    piece_list = pickle.load(f)
    
figsize1 = (10, 3)  
dpi1 = 200         
use_window = False
onset_types = ["Dun", "J1", "J2"]
save_dir = "output_dot_plots"
    
for file_name in piece_list:
    print(file_name)
    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
    # bothfeet_onset_path = f"data/dance_onsets/{file_name}_T_dance_onsets.csv"
    dmode_path = f"data/dance_modes_ts/{file_name}_{mode[m_idx]}.pkl"
    
    left_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_left_foot_onsets.csv"
    right_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_right_foot_onsets.csv"
    
    left_onsets = pd.read_csv(left_onset_path)["time_sec"].values
    right_onsets = pd.read_csv(right_onset_path)["time_sec"].values
    
    print(left_onsets)
    print(right_onsets) 
        
    if os.path.exists(dmode_path):
        print(dmode_path)
        with open(dmode_path, "rb") as f:
            dance_mode_time_segments = pickle.load(f)       # list of tuples (start_time, end_time)
            print(dance_mode_time_segments)
    else:
        continue
    
    # start_time, end_time = dance_mode_time_segments[0]


    for onset_type in onset_types:
  
        # have to change this part of the code
        if not use_window:
            save_path = f"output_static_plot\\{file_name}_{onset_type}_full_duration_subplot.png"
        else:
            save_path = os.path.join(save_dir, f"{file_name}_{onset_type}_{W_start}_{W_end}.png")
        
        # Analyze phases and save plot
        analyze_phases(
            cycles_csv_path, onsets_csv_path, onset_type, W_start, W_end,
            save_path=save_path, figsize=figsize1, dpi=dpi1,
            use_window=use_window
        )

        
        print(f"Saved plot for {onset_type} to {save_path}")



BKO_E1_D1_01_Suku
[ 34.25833333  36.27916667  38.3         40.3125      42.3
  44.3         46.27916667  48.2125      50.2125      52.125
  54.075       55.97083333  57.8875      59.80833333  61.6875
  63.57916667  65.44583333  67.25833333  69.125       70.95
  72.83333333  74.62083333  80.39166667  97.7875      99.22916667
 102.49583333 104.43333333 111.67916667 116.41666667 118.35
 119.59166667 120.77083333 122.1375     123.34583333 124.74583333
 126.         127.3625     129.29583333 130.55416667 131.11666667
 133.01666667 133.61666667 135.51666667 136.13333333 138.00833333
 138.65       140.47916667 141.10416667 142.4        143.0875
 143.8125     145.05       145.50416667 145.94583333 146.23333333
 146.65833333 147.10833333 147.4375     147.85833333 148.32916667
 148.65416667 149.04166667 149.52083333 149.8375     150.23333333
 150.69166667 151.02916667 151.925      152.64166667 155.1
 155.93333333 156.95833333 157.17083333 158.18333333 158.40833333
 158.97083333 159.45833333 159.

In [ ]:
# dance_csv_path = f"data/dance_onsets/{filename}_T_dance_onsets.csv"
file_list = os.listdir(f"data/subset_dance_annotation")

ls_virtual_cycles = os.listdir(f"data/virtual_cycles")
ls_drum_onsets = os.listdir(f"data/drum_onsets")



for file_name in file_list:
    file_name = file_name.split("_Dancers")[0]

    cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
    onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
    dmode_path = f"data/dance_modes_ts/{file_name}_group.pkl"
    danceOnset_path = f"data/dance_onsets/{file_name}_T_dance_onsets.csv"
    
    print(dmode_path)
    if os.path.exists(dmode_path):
        
        with open(dmode_path, "rb") as f:
            dmode = pickle.load(f)
    else:
        continue
    
    start_time, end_time = dmode[0]
    
    save_dance_phase_plot(
        file_name= file_name,
        W_start=start_time,
        W_end=end_time,
        cycles_csv_path=cycles_csv_path,
        dance_csv_path= danceOnset_path,
        save_dir="output_static_plot\dance_group",
        save_format="png"  # or 'jpg', 'pdf', etc.
)

### drum

In [ ]:
import pickle

ls_virtual_cycles = os.listdir(f"data/virtual_cycles")
ls_drum_onsets = os.listdir(f"data/drum_onsets")
mode = ["group", "individual", "audience"]
m_idx = 0


for file_name in ls_drum_onsets:
    piece = file_name.split("_")[-1].lower()[:-4]
    fname = file_name.split(".")[0]
    if piece in boundary_lines.keys(): 
        print(piece, fname)

        cycles_csv_path = f"data/virtual_cycles/{fname}_C.csv"
        onsets_csv_path = f"data/drum_onsets/{fname}.csv"
        dmode_path = f"data/dance_modes_ts/{file_name[:-4]}_group.pkl"
        danceOnset_path = f"data/dance_onsets/{file_name[:-4]}_T_dance_onsets.csv"
        
        print(dmode_path)
        if os.path.exists(dmode_path):
            
            with open(dmode_path, "rb") as f:
                dmode = pickle.load(f)
        else:
            continue
        
        start_time, end_time = dmode[0]
        fig, _ = plot_merged_stacked(fname,
                                cycles_csv_path,
                                onsets_csv_path,
                                boundary_lines[piece],
                                W_start=start_time,
                                W_end=end_time,
                                figsize= (10, 6),
                                dpi=300,
                                use_window=False,
                                )
        save_path_full = f"output_static_plot\\drum_group\\{fname}_group.png"
        fig.savefig(save_path_full, bbox_inches='tight', dpi=300)
